# Previous working script

In [4]:
from pymatgen.ext.matproj import MPRester

In [6]:
import mlstabilitytest
import automatminer

In [48]:
import os
from mlstabilitytest.stability.StabilityAnalysis import StabilityAnalysis, EdAnalysis
from shutil import copyfile

here = './mlstabilitytest/'

models = ['ElFrac', 'Meredig', 'Magpie', 'AutoMat', 'ElemNet', 'Roost', 
              'CGCNN']
experiments = ['LiMnTMO', 'allMP', 'smact',
               'random1', 'random2', 'random3',
               'classifier']
training_props = ['Ef', 'Ed']
path_to_ml_data = os.path.join(here, 'ml_data')

for training_prop in training_props:
    print('\n____ models trained on %s ____\n' % training_prop)
    for experiment in experiments:
        print('\n ~~~ %s ~~~\n' % experiment)
        experiment_dir = os.path.join(path_to_ml_data, training_prop, experiment)
        if (('random' not in experiment) and (training_prop == 'Ed')) or ((experiment != 'classifier') and (training_prop == 'Ef')):
            if not os.path.exists(experiment_dir):
                os.mkdir(experiment_dir)
        for model in models:
            print('\n %s ' % model)
            process(training_prop, model, experiment, path_to_ml_data)
            
def process(training_prop, model, experiment, path_to_ml_data):
    """
    Args:
        training_prop (str) - 'Ef' if models trained on formation energies; 'Ed' if decomposition energies
        model (str) - ML model
        experiment (str) - 'allMP', 'LiMnTMO', or 'smact'
        path_to_ml_data (os.PathLike) - path to ml_data directory in .../TestStabilityML/mlstabilitytest/ml_data
    
    Returns:
        Runs all relevant analyses
        Prints a summary
    """
    if (model == 'CGCNN') and (experiment == 'smact'):
        print('CGCNN cannot be applied directly to the SMACT problem because the structures are not known')
        return
    if ('random' in experiment) and (training_prop == 'Ed'):
        print('Random perturbations only apply to models trained on Ef as written')
        return
    if (experiment == 'classifier') and (training_prop == 'Ef'):
        print('Classifier experiment only applies to training on Ed')
        return
    if (model == 'CGCNN') and (training_prop == 'Ed'):
        print('CGCNN not trained on Ed')
        return
    data_dir = os.path.join(path_to_ml_data, training_prop, experiment, model)
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
        
    data_file = 'ml_input.json'
    finput = os.path.join(data_dir, data_file)
    if 'random' in experiment:
        src = finput.replace(experiment, 'allMP')
        copyfile(src, finput)
    if not os.path.exists(finput):
        print('missing data for %s-%s' % (model, experiment))
        return
    if training_prop == 'Ef':
        nprocs = 'all'
        obj = StabilityAnalysis(data_dir,
                                data_file,
                                experiment,
                                nprocs=nprocs)
    elif training_prop == 'Ed':
        obj = EdAnalysis(data_dir,
                         data_file,
                         experiment)
    else:
        raise NotImplementedError
    obj.results_summary
    print('got results')
    return


____ models trained on Ef ____


 ~~~ LiMnTMO ~~~



FileNotFoundError: [Errno 2] No such file or directory: '/home/andi/TestStabilityML/ml_data/Ef/LiMnTMO'

# Testing working script 

In [40]:
import os
from mlstabilitytest.stability.StabilityAnalysis import StabilityAnalysis, EdAnalysis
from shutil import copyfile

here = './mlstabilitytest/'
path_to_ml_data = os.path.join(here, 'ml_data')

model = 'Magpie'
experiment = 'allMP'
training_prop = 'Ed'

process(training_prop, model, experiment, path_to_ml_data)


Checking input data...
Data looks good.
Time elapsed = 5 s.

Reading existing results file: ./mlstabilitytest/ml_data/Ed/allMP/Magpie/ml_results.json

Summarizing performance...

MAE on decomposition enthalpy = 0.092 eV/atom


Classifying stable or unstable:
Precision = 0.714
Recall = 0.379
Accuracy = 0.691
F1 = 0.495
FPR = 0.101

Confusion matrix:
TP | FP
FN | TN = 
12886 | 5160
21112 | 45856

Time elapsed = 0 s
got results


# Testing script for decomposition only

In [39]:
import os
from mlstabilitytest.stability.StabilityAnalysis import EdAnalysis
from shutil import copyfile

here = './mlstabilitytest/'

# models = ['ElFrac', 'Meredig', 'Magpie', 'AutoMat', 'ElemNet', 'Roost', 
#               'CGCNN']
model = 'Magpie'

experiment = 'LiMnTMO'
training_props = 'Ed'
path_to_ml_data = os.path.join(here, 'ml_data')


data_dir = os.path.join(path_to_ml_data, training_prop, experiment, model)
data_file = 'ml_input.json'

obj = EdAnalysis(data_dir, data_file, experiment)

obj.results_summary


Checking input data...
Data looks good.
Time elapsed = 0 s.

Reading existing results file: ./mlstabilitytest/ml_data/Ef/LiMnTMO/Magpie/ml_results.json

Summarizing performance...

MAE on decomposition enthalpy = 0.052 eV/atom


Classifying stable or unstable:
Precision = 0.036
Recall = 0.111
Accuracy = 0.869
F1 = 0.054
FPR = 0.105

Confusion matrix:
TP | FP
FN | TN = 
1 | 27
8 | 231

Time elapsed = 0 s
